In [17]:
# imports
import os
import glob
from dotenv import load_dotenv
import gradio as gr


In [18]:
# imports for langchain, plotly and Chroma

from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np
import plotly.graph_objects as go
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import HuggingFaceEmbeddings


In [19]:
# price is a factor for our company, so we're going to use a low cost model

MODEL = "gpt-4o-mini"
db_name = "vector_db"


In [20]:
# Load environment variables in a file called .env

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')


In [21]:
# Read in documents using LangChain's loaders
# Take everything in all the sub-folders of our knowledgebase

import glob
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import PyMuPDFLoader
from langchain.document_loaders import UnstructuredPDFLoader


# Replace with your actual folder path where PDFs are stored
pdf_files = glob.glob("DB/*", recursive=True)

def add_metadata(doc, doc_type):
    doc.metadata["doc_type"] = doc_type
    return doc

# With thanks to CG and Jon R, students on the course, for this fix needed for some users 
text_loader_kwargs = {'encoding': 'utf-8'}
# If that doesn't work, some Windows users might need to uncomment the next line instead
# text_loader_kwargs={'autodetect_encoding': True}

all_documents = []

for pdf_path in pdf_files:
    doc_type = os.path.basename(pdf_path)
    loader = PyPDFLoader(pdf_path)
    pages = loader.load()
    all_documents.extend(pages)

text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=80)
chunks = text_splitter.split_documents(all_documents)

print(f"Chunk size: {text_splitter._chunk_size}, Overlap: {text_splitter._chunk_overlap}")



Ignoring wrong pointing object 0 0 (offset 0)


Chunk size: 500, Overlap: 80


In [22]:
print(f"Total number of chunks: {len(chunks)}")


Total number of chunks: 521


In [23]:
# Put the chunks of data into a Vector Store that associates a Vector Embedding with each chunk
# Chroma is a popular open source Vector Database based on SQLLite

load_dotenv()
embeddings = OpenAIEmbeddings()

# If you would rather use the free Vector Embeddings from HuggingFace sentence-transformers
# Then replace embeddings = OpenAIEmbeddings()
# with:
# from langchain.embeddings import HuggingFaceEmbeddings
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")



# Create vectorstore

vectorstore = Chroma.from_documents(chunks, embedding=embeddings)


llm = ChatOpenAI(temperature=0.7, model_name=MODEL)

# Alternative - if you'd like to use Ollama locally, uncomment this line instead
# llm = ChatOpenAI(temperature=0.7, model_name='llama3.2', base_url='http://localhost:11434/v1', api_key='ollama')

memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# the retriever is an abstraction over the VectorStore that will be used during RAG
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# putting it together: set up the conversation chain with the GPT 3.5 LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

C:\Users\ibraa\AppData\Local\Temp\ipykernel_2152\22356074.py:25: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)


In [24]:
# Let's try a simple question

query = "explain what ACL loans are"
result = conversation_chain.invoke({"question": query})
print(result["answer"])

ACL stands for "Allowance for Credit Losses" on loans and leases. It is the term used by banks that have adopted ASU 2016-13, which implements ASC Topic 326, Financial Instruments – Credit Losses. The ACL represents a valuation account that is used to present the net amount expected to be collected over the contractual term of the financial assets, considering expected prepayments. Essentially, the ACL is set aside to absorb estimated credit losses associated with a bank's loan and lease portfolio. It reflects management's expectations about future credit losses and is evaluated at the end of each reporting period.


In [26]:
# Put the chunks of data into a Vector Store that associates a Vector Embedding with each chunk
# Chroma is a popular open source Vector Database based on SQLLite
from langchain.memory import ConversationTokenBufferMemory  # ✅ ADD THIS IMPORT

load_dotenv()
embeddings = OpenAIEmbeddings()

# If you would rather use the free Vector Embeddings from HuggingFace sentence-transformers
# Then replace embeddings = OpenAIEmbeddings()
# with:
# from langchain.embeddings import HuggingFaceEmbeddings
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")



# Create vectorstore

vectorstore = Chroma.from_documents(chunks, embedding=embeddings)


llm = ChatOpenAI(temperature=0.7, model_name=MODEL)

# Alternative - if you'd like to use Ollama locally, uncomment this line instead
# llm = ChatOpenAI(temperature=0.7, model_name='llama3.2', base_url='http://localhost:11434/v1', api_key='ollama')

# set up the conversation memory for the chat
memory = ConversationTokenBufferMemory(
    llm=llm,
    max_token_limit=1000,  # adjust based on your model’s context window
    memory_key='chat_history',
    return_messages=True
)
# the retriever is an abstraction over the VectorStore that will be used during RAG
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})


# putting it together: set up the conversation chain with the GPT 3.5 LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

C:\Users\ibraa\AppData\Local\Temp\ipykernel_2152\1192372031.py:27: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationTokenBufferMemory(


In [27]:
def chat(question, history):
    result = conversation_chain.invoke({"question": question})
    return result["answer"]

In [28]:
view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
